In [ ]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


# 네이버 뉴스기사 크롤링 (지난 일주일 간)

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import urllib.parse
import time
import pandas as pd
from datetime import datetime, timedelta

#####################################################
# 검색어 및 날짜 설정
text = "트럼프"  # 검색어 입력
base_date = "2025-03-03"  # 기준 날짜
#####################################################

# 날짜 변환
base_date_dt = datetime.strptime(base_date, "%Y-%m-%d")

# 하루씩 빼면서 일주일치 날짜 리스트 생성
day_list = [(base_date_dt - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(1, 8)]

# ChromeDriver 경로 설정
driver_path = "/Users/leesangwon/Desktop/chromedriver-mac-arm64/chromedriver"  # 크롬드라이버 경로 입력
service = Service(driver_path)

# 웹드라이버 초기화
driver = webdriver.Chrome(service=service)

# 전체 데이터 저장 리스트
all_data = []

for date in day_list:
    y, m, d = date.split("-")  # 연, 월, 일 분리

    # URL 생성
    encoded_text = urllib.parse.quote(text)  # 검색어 URL 인코딩
    url = f"https://search.naver.com/search.naver?where=news&query={encoded_text}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={y}.{m}.{d}&de={y}.{m}.{d}&nso=so%3Ar%2Cp%3Afrom{y}{m}{d}to{y}{m}{d}"

    # 페이지 열기
    driver.get(url)
    time.sleep(1.5)  # 페이지 로딩 대기

    # 기사 개수를 체크하면서 스크롤 조절
    scroll_count = 0
    while scroll_count < 5:  # 최대 5회까지만 스크롤
        titles = driver.find_elements(By.CSS_SELECTOR, ".news_tit")
        if len(titles) >= 30:  # 30개 이상이면 스크롤 중단
            break
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.2)
        scroll_count += 1

    # 기사 제목, 날짜, 언론사, 내용 크롤링
    titles = driver.find_elements(By.CSS_SELECTOR, ".news_tit")
    dates = driver.find_elements(By.CSS_SELECTOR, ".info_group > span")
    scripts = driver.find_elements(By.CSS_SELECTOR, ".dsc_txt_wrap")
    press = driver.find_elements(By.CSS_SELECTOR, ".info_group > a.press")

    # 하루 단위로 데이터 저장
    day_data = []
    for title, date, script, p in zip(titles[:30], dates[:30], scripts[:30], press[:30]):
        day_data.append({
            "Title": title.text,
            "Date": date.text,
            "Press": p.text,
            "Description": script.text
        })

    all_data.extend(day_data)  # 전체 데이터에 추가

# 데이터프레임 생성 및 CSV 저장
df = pd.DataFrame(all_data)
csv_filename = f"/Users/leesangwon/Downloads/{text}_{day_list[-1]}_to_{day_list[0]}_news_data.csv"
df.to_csv(csv_filename, index=False)

# 드라이버 종료
driver.quit()

print(f"✅ 크롤링 완료: {csv_filename} 저장됨.")
time.sleep(3)

✅ 크롤링 완료: /Users/leesangwon/Downloads/트럼프_2025-02-24_to_2025-03-02_news_data.csv 저장됨.


# 형태소 분석 및 토크나이징

In [41]:
import nltk
import pandas as pd
from nltk.probability import FreqDist
from konlpy.tag import Kkma

# 파일명 지정
news_data = "딥시크_2025-02-23_to_2025-03-01_news_data"
csv_filename = f"/Users/leesangwon/Downloads/{news_data}.csv"  # CSV 파일 경로

# CSV 파일 로드
df = pd.read_csv(csv_filename, encoding="utf-8-sig")  

# "Title"과 "description"을 합쳐서 하나의 문자열로 변환
df = df.dropna(subset=["Title", "Description"])  # NaN 제거
text = " ".join(df["Title"].astype(str) + " " + df["Description"].astype(str))

# Kkma 형태소 분석기 초기화
kkma = Kkma()

# 형태소 분석 후 명사(NNG, NNP) 추출
morphs = kkma.pos(text)
nouns = [word for word, tag in morphs if tag in ["NNG", "NNP"]]

# NLTK를 이용하여 단어 빈도수 분석
nltk.download("punkt")  # 첫 실행 시 필요
freq_dist = FreqDist(nouns)

# 데이터프레임 생성
df_freq = pd.DataFrame(freq_dist.items(), columns=["단어", "빈도수"])

# CSV 파일 저장
output_csv_filename = f"/Users/leesangwon/Downloads/freq_{news_data}.csv"
df_freq.to_csv(output_csv_filename, index=False, encoding="utf-8-sig")  # utf-8-sig로 한글 깨짐 방지

print(f"✅ CSV 파일 저장 완료: {output_csv_filename}")

✅ CSV 파일 저장 완료: /Users/leesangwon/Downloads/freq_딥시크_2025-02-23_to_2025-03-01_news_data.csv


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leesangwon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
